In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from xgboost import XGBRFRegressor


In [2]:
#loading the data

X_train_full = pd.read_csv("train.csv", index_col= "id")
X_training_extra_full = pd.read_csv("training_extra.csv", index_col= "id")
X_test_full = pd.read_csv("test.csv", index_col= "id")


In [3]:
#setting the target and features

y = X_train_full["Price"]
X = X_train_full.drop("Price", axis = 1)

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size= 0.8, test_size= 0.2, random_state= 0)

In [4]:
X_train.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
id,,,,,,,,,
154452,Under Armour,Polyester,Small,8.0,No,Yes,Backpack,Red,16.817078
297220,NaN,Leather,Medium,10.0,NaN,Yes,Backpack,Blue,16.750179
177,Under Armour,Nylon,Medium,8.0,Yes,Yes,Messenger,Green,13.262331
44603,Adidas,Polyester,Small,10.0,No,No,Tote,Gray,29.694404
158177,Puma,Nylon,Small,2.0,No,Yes,Messenger,Blue,11.291522


In [5]:
#getting all obj columns in the data

obj_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
obj_cols


['Brand',
 'Material',
 'Size',
 'Laptop Compartment',
 'Waterproof',
 'Style',
 'Color']

In [6]:
#Getting all unique values found in the Object/Categorical columns

unique_labels = [X_train[col].unique() for col in X_train[obj_cols]]

for i in unique_labels:
    print(i)

['Under Armour' nan 'Adidas' 'Puma' 'Jansport' 'Nike']
['Polyester' 'Leather' 'Nylon' nan 'Canvas']
['Small' 'Medium' 'Large' nan]
['No' nan 'Yes']
['Yes' 'No' nan]
['Backpack' 'Messenger' 'Tote' nan]
['Red' 'Blue' 'Green' 'Gray' 'Pink' nan 'Black']


In [7]:
#We select all the columns from the df that are not object type 

cat_X_train = X_train[obj_cols]
cat_X_val = X_val[obj_cols]

num_X_train = X_train.select_dtypes(exclude = ["object"])
num_X_val = X_val.select_dtypes(exclude = ["object"])

#numerical_X_train
#cat_X_train

In [8]:
#checking how many missing values there are in the data
X_train.isnull().sum()

Brand                   7767
Material                6693
Size                    5274
Compartments               0
Laptop Compartment      5938
Waterproof              5697
Style                   6399
Color                   7972
Weight Capacity (kg)     110
dtype: int64

In [9]:
cat_imputer = SimpleImputer(strategy= "most_frequent")
cat_imputer.fit_transform(pd.DataFrame(X_train[obj_cols]))

num_inputer = SimpleImputer(strategy= "mean")
num_X_train = pd.DataFrame(num_inputer.fit_transform((num_X_train)))


In [10]:
#One hot incoding the categorical columns for train and validaiton data
OH_imputer = OneHotEncoder(handle_unknown="ignore", sparse_output= False)

OH_X_train_cols = pd.DataFrame(OH_imputer.fit_transform(cat_X_train))
OH_X_val_cols = pd.DataFrame(OH_imputer.transform(cat_X_val))


#putting the index back after OH incoding gets rid of them
OH_X_train_cols.index = X_train.index
OH_X_val_cols.index = X_val.index

#insuring the columns are "str" dtype
OH_X_train_cols.columns = OH_X_train_cols.columns.astype("str")
OH_X_val_cols.columns = OH_X_val_cols.columns.astype("str")

#putting the new OH encoded columns together with the numerical ones
X_train_ready = pd.concat([num_X_train, OH_X_train_cols], axis = 1)
X_val_ready = pd.concat([num_X_val, OH_X_val_cols], axis = 1)



In [12]:
model = XGBRFRegressor()

model.fit(X_train_ready, y_train)

#predictions = model.predict(X_val)



XGBoostError: [06:29:32] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\data\data.cc:508: Check failed: this->labels.Size() % this->num_row_ == 0 (240000 vs. 0) : Incorrect size for labels.